In [ ]:
pip install requests

In [ ]:
import requests
import pandas as pd

In [ ]:
url = "	http://data.ex.co.kr/openapi/odtraffic/trafficAmountByUnit"

In [ ]:
key = "0113111078"

In [ ]:
params = {'key':key, 'type':'xml', 'subTmUnitTypeCode': '3', 'stdHour': '07', 'numOfRows':'1000', 'pageNo':'1'}

In [ ]:
response = requests.get(url,params=params)

In [ ]:
response.content

In [ ]:
print(response.apparent_encoding)

In [ ]:
type(response.content)

In [ ]:
data=str(response.content, "utf-8")
data

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
tree = ET.fromstring(data)
tree

In [ ]:
len(tree)

In [ ]:
list(tree)

In [ ]:
new_data = tree[5]
print(len(new_data))
print(new_data)

In [ ]:
for i in range(len(new_data)):
  print(new_data[i].tag, new_data[i].text, i)

In [ ]:
for a in range(3,102):
  new_data = tree[a]
  print(new_data[1].text, new_data[3].text, new_data[4].text,
        new_data[5].text, new_data[6].text, new_data[7].text,
        new_data[8].text, new_data[9].text, new_data[10].text,
        new_data[12].text, new_data[13].text, new_data[11].tag, new_data[11].text)
  

In [ ]:
df=[]
for a in range(3,102):
  temp = []
  # 4번(입구출구), 11번(톨게이트이름), 13번(교통량) 열만 사용
  for b in [4,11,13]:
    temp.append(tree[a][b].text)
  df.append(temp)
df

In [ ]:
df = pd.DataFrame(df)
df.head()

In [ ]:
df.columns = ['exit', 'volume', 'toll']
df.head(3)

In [ ]:
df.info()

In [ ]:
df['volume'] = df['volume'].astype(int)

In [ ]:
df = df[df.iloc[:,0] == '출구']
df

In [ ]:
traffic = df.groupby('toll')['volume'].sum()
df_traffic = pd.DataFrame(traffic)
df_traffic

In [ ]:
import plotly.express as px

fig = px.bar(
    df_traffic, 
    y="volume",
    width=800,
    height=450,
    title='톨게이트별 교통량'
)
fig.show()

In [ ]:
params = {'key' : key, 'type': 'xml', 'sumTmUnitTypeCode' : '3', 'stdHour': '07', 'numOfRows':'1000'}
response = requests.get(url,params=params)
data=str(response.content,"utf-8")
data

In [ ]:
tree = ET.fromstring(data)
tree

In [ ]:
for a in range(30):
  new_data = tree[a]
  for b in range (len(new_data)):
    print(new_data[b].text, end=' ')
  print()

In [ ]:
df = []
for a in range(3, 25):
  temp = []
  # 4번(입구출구), 6번(기준시), 10번(차중), 11번(톨게이트), 13번(교통량) 열만 사용
  for b in [4, 6, 10, 11, 13]:
    temp.append(tree[a][b].text)
  df.append(temp)
df

In [ ]:
df7 = pd.DataFrame(df)
df7.head(3)

In [ ]:
def hourly(hour):
  params = {'key':key, 'type':'xml', 'sumTmUnitTypeCode' : '3', 'stdHour': hour, 'numOfRows': '1000', 'pageNO': 1}
  response = requests.get(url, params=params)
  data = str(response.content, "utf-8")
  tree = ET.fromstring(data)
  df=[]

  for a in range(3,25):
    temp = []
    # 4번(입구출구), 6번(기준시), 10번(차중), 11번(톨게이트), 13번(교통량) 열만 사용
    for b in [4, 6, 10, 11, 13]:
      temp.append(tree[a][b].text)
    df.append(temp)
  df = pd.DataFrame(df)

  return df


In [ ]:
df5 = hourly('05')
df5.head(3)

In [ ]:
df6 = hourly('06')
df6.head(3)

In [ ]:
df7 = hourly('07')
df7.head(3)

In [ ]:
df8 = hourly('08')
df8.head(3)

In [ ]:
df = pd.concat([df5, df6, df7, df8])
df

In [ ]:
df.columns = ['exit','hour', 'type', 'volume', 'toll']
df.head(5)

In [ ]:
df['volume'] = df['volume'].astype(int)

In [ ]:
df_dangjin = df[(df.loc[ :, 'exit'] == '출구') & (df.loc[ :, 'toll'] == '당진')]
df_dangjin

In [ ]:
traffic1 = df.groupby('hour')['volume'].sum()
df1_traffic = pd.DataFrame(traffic1)

df1_traffic

In [ ]:
import plotly.express as px

fig = px.bar(
    df1_traffic,
    y='volume',
    width=800,
    height=450,
    title='당진 톨게이트 시간대별 교통량'
)
fig.show()